[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/romiaprilian7406/sp500-relative-valuation-estimation/blob/main/notebooks/sp500_relative_valuation_dataset.ipynb)


# Import Library

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm.auto import tqdm
import time
import warnings

warnings.filterwarnings('ignore')

# Global Configuration

In [2]:
DATA_URL = "https://raw.githubusercontent.com/romiaprilian7406/sp500-companies/main/data/sp500_companies.csv"
EXCLUDED_SECTORS = ['Financials', 'Real Estate'] # Sektor dengan struktur neraca berbeda
REQUEST_DELAY = 0.5 # Detik, untuk menghindari rate limit Yahoo Finance

# Ticker Management

In [3]:
# Mengunduh daftar S&P 500, membuang sektor yang tidak diinginkan, dan memperbaiki format simbol ticker
def load_tickers(url, excluded_sectors):
    print(f"Mengunduh list ticker S&P500 dari: {url}")
    df = pd.read_csv(url)

    initial_count = len(df)

    # Filter Sektor
    df_filtered = df[~df['GICS Sector'].isin(excluded_sectors)].copy()

    # Format Symbol: Ubah 'BRK.B' menjadi 'BRK-B' agar terbaca yfinance
    df_filtered['Symbol'] = df_filtered['Symbol'].str.replace('.', '-', regex=False)

    # Logging sederhana
    dropped_count = initial_count - len(df_filtered)
    print(f"Total Awal    : {initial_count}")
    print(f"Dibuang       : {dropped_count} (Sectors: {excluded_sectors})")
    print(f"Total Target  : {len(df_filtered)}")

    return df_filtered

# Membuat dictionary mapping {Ticker: Sector} untuk referensi cepat
def get_sector_map(df):
    return dict(zip(df['Symbol'], df['GICS Sector']))

# Data Extraction

In [4]:
# Helper internal untuk mengambil nilai float secara aman dari Dictionary atau Series.
# Mengembalikan np.nan jika tidak ditemukan atau error
def safe_get(source, keys):
    # Jika source adalah dictionary (misal: stock.info)
    if isinstance(source, dict):
        val = source.get(keys, np.nan)
        return float(val) if val is not None else np.nan

    # Jika source adalah Pandas Series (misal: financials)
    if isinstance(keys, str):
        keys = [keys]

    for k in keys:
        if k in source:
            val = source[k]
            # Cek validitas data sebelum convert
            return float(val) if not pd.isna(val) else np.nan

    return np.nan

# Mengambil data fundamental lengkap untuk satu ticker menggunakan yfinance
def fetch_financial_data(ticker, sector):
    try:
        stock = yf.Ticker(ticker)

        # 1. Fetch .info (Market Data & Estimates)
        info = stock.info

        # 2. Fetch Financial Statements (Fail-safe)
        try:
            # Mengambil kolom pertama (tahun/kuartal terbaru)
            inc = stock.financials.iloc[:, 0] if not stock.financials.empty else pd.Series(dtype=float)
            bal = stock.balance_sheet.iloc[:, 0] if not stock.balance_sheet.empty else pd.Series(dtype=float)
            cf = stock.cashflow.iloc[:, 0] if not stock.cashflow.empty else pd.Series(dtype=float)
        except Exception:
            # Jika gagal ambil statement, set kosong tapi tetap lanjut ambil data .info
            inc, bal, cf = pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)

        # 3. Construct Data Dictionary
        data = {
            'Ticker': ticker,
            'Sector': sector,

            # TARGET VARIABLES
            'EnterpriseValue': safe_get(info, 'enterpriseValue'),
            'EBITDA': safe_get(inc, ['EBITDA', 'Normalized EBITDA']),

            # MODEL FEATURES (Growth & Risk)
            'ForwardPE': safe_get(info, 'forwardPE'),
            'TrailingPE': safe_get(info, 'trailingPE'),
            'Beta': safe_get(info, 'beta'),
            'TotalRevenue': safe_get(inc, 'Total Revenue'),

            # ROIC COMPONENTS (Quality)
            'EBIT': safe_get(inc, ['EBIT', 'Operating Income', 'Operating Profit']),
            'PretaxIncome': safe_get(inc, ['Pretax Income', 'Income Before Tax']),
            'TaxProvision': safe_get(inc, ['Tax Provision', 'Income Tax Expense']),

            # BALANCE SHEET (Invested Capital)
            'TotalAssets': safe_get(bal, 'Total Assets'),
            'TotalEquity': safe_get(bal, ['Stockholders Equity', 'Total Equity Gross Minority Interest']),
            'TotalDebt': safe_get(bal, ['Total Debt', 'Total Liab']),
            'CashAndEquivalents': safe_get(bal, ['Cash And Cash Equivalents', 'Cash', 'Cash Financial'])
        }

        return data

    except Exception:
        return None

# Execution

In [5]:
# Load Tickers
df_sp500 = load_tickers(DATA_URL, EXCLUDED_SECTORS)
tickers = df_sp500['Symbol'].tolist()
sector_map = get_sector_map(df_sp500)

# Processing Loop
results = []
print(f"\nMemulai pengambilan data untuk {len(tickers)} saham")

# Menggunakan tqdm untuk progress bar
for ticker in tqdm(tickers, desc="Fetching Data"):

    sector = sector_map.get(ticker, 'Unknown')
    data = fetch_financial_data(ticker, sector)

    if data:
        results.append(data)

    # Rate Limiting
    time.sleep(REQUEST_DELAY)

print(f"Proses Selesai, {len(results)} data berhasil diambil")

Mengunduh list ticker S&P500 dari: https://raw.githubusercontent.com/romiaprilian7406/sp500-companies/main/data/sp500_companies.csv
Total Awal    : 503
Dibuang       : 107 (Sectors: ['Financials', 'Real Estate'])
Total Target  : 396

Memulai pengambilan data untuk 396 saham


Fetching Data:   0%|          | 0/396 [00:00<?, ?it/s]

Proses Selesai, 396 data berhasil diambil


In [6]:
if results:
    df_final = pd.DataFrame(results)

    # Tampilkan preview
    print("\nPreview data:")
    display(df_final.head())
else:
    print("GAGAL: Tidak ada data")


Preview data:


,Ticker,Sector,EnterpriseValue,EBITDA,ForwardPE,TrailingPE,Beta,TotalRevenue,EBIT,PretaxIncome,TaxProvision,TotalAssets,TotalEquity,TotalDebt,CashAndEquivalents
0,MMM,Industrials,9.418677e+10,7.373000e+09,18.855309,25.953821,1.148,2.457500e+10,6.010000e+09,4.819000e+09,8.040000e+08,3.986800e+10,3.842000e+09,1.365900e+10,5.600000e+09
1,AOS,Industrials,9.529010e+09,7.865000e+08,16.587528,18.446090,1.335,3.818100e+09,7.077000e+08,7.010000e+08,1.674000e+08,3.240000e+09,1.883500e+09,2.167000e+08,2.396000e+08
2,ABT,Health Care,2.258845e+11,1.079000e+10,22.362432,15.944723,0.714,4.195000e+10,7.572000e+09,7.013000e+09,-6.389000e+09,8.141400e+10,4.766400e+10,1.502100e+10,7.616000e+09
3,ABBV,Health Care,4.596630e+11,1.491000e+10,15.794969,170.545440,0.349,5.633400e+10,6.524000e+09,3.716000e+09,-5.700000e+08,1.351610e+11,3.325000e+09,6.714400e+10,5.524000e+09
4,ACN,Information Technology,1.675005e+11,1.186733e+10,17.972982,21.984774,1.261,6.967298e+10,1.049895e+10,1.027039e+10,2.437993e+09,6.539490e+10,3.119545e+10,8.182866e+09,1.147873e+10


# Simple EDA

In [7]:
# Shape
rows, cols = df_final.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# Info
print(df_final.info(),"\n")

# Fitur adalah semua kolom selain Target dan Metadata
features_list = [col for col in df_final.columns]

print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# Statistik Deskriptif Fitur Numerik
print(f"\nStatistik Deskriptif")
numeric_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()

try:
    display(df_final[numeric_cols].describe().T.round(2))
except:
    print(df_final[numeric_cols].describe().T.round(2))

# Missing Values
print(f"\nLaporan Missing Values:")
missing_count = df_final.isnull().sum()
missing_pct = (df_final.isnull().sum() / len(df_final)) * 100

missing_df_final = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct
})

# Filter hanya yang ada missing value & Sort dari yang terbanyak
missing_only = missing_df_final[missing_df_final['Jumlah Hilang'] > 0].sort_values(by='Jumlah Hilang', ascending=False)

# Format persentase setelah sorting
missing_only['Persentase (%)'] = missing_only['Persentase (%)'].map('{:.2f}%'.format)

if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values")

Dimensi Dataset
Total Baris    : 396
Total Kolom    : 15

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              396 non-null    object 
 1   Sector              396 non-null    object 
 2   EnterpriseValue     393 non-null    float64
 3   EBITDA              391 non-null    float64
 4   ForwardPE           396 non-null    float64
 5   TrailingPE          373 non-null    float64
 6   Beta                390 non-null    float64
 7   TotalRevenue        394 non-null    float64
 8   EBIT                394 non-null    float64
 9   PretaxIncome        394 non-null    float64
 10  TaxProvision        393 non-null    float64
 11  TotalAssets         396 non-null    float64
 12  TotalEquity         396 non-null    float64
 13  TotalDebt           393 non-null    float64
 14  CashAndEquivalents  396 non-null    float64
dtyp

,count,mean,std,min,25%,50%,75%,max
EnterpriseValue,393.0,1.482512e+11,4.556311e+11,3.747220e+09,2.477422e+10,4.802480e+10,1.066581e+11,4.104056e+12
EBITDA,391.0,7.576849e+09,1.795178e+10,-7.649000e+09,1.559950e+09,2.829500e+09,6.446804e+09,1.601650e+11
ForwardPE,396.0,2.108000e+01,2.078000e+01,-1.330500e+02,1.373000e+01,1.865000e+01,2.432000e+01,2.132700e+02
TrailingPE,373.0,3.763000e+01,5.513000e+01,5.010000e+00,1.948000e+01,2.585000e+01,3.498000e+01,6.043300e+02
Beta,390.0,9.700000e-01,4.300000e-01,-8.000000e-02,6.400000e-01,9.900000e-01,1.250000e+00,2.540000e+00
TotalRevenue,394.0,3.806347e+10,7.653456e+10,7.058230e+08,7.079825e+09,1.465271e+10,3.063975e+10,6.809850e+11
EBIT,394.0,5.499791e+09,1.466203e+10,-1.017600e+10,1.080575e+09,2.088500e+09,4.705525e+09,1.330500e+11
PretaxIncome,394.0,4.929216e+09,1.446431e+10,-1.221000e+10,9.102500e+08,1.729500e+09,4.121250e+09,1.327290e+11
TaxProvision,393.0,9.168030e+08,2.540082e+09,-6.389000e+09,1.431000e+08,3.230000e+08,7.614610e+08,2.179500e+10
TotalAssets,396.0,5.170993e+10,7.991535e+10,1.248020e+09,1.224000e+10,2.382730e+10,5.493150e+10,6.248940e+11



Laporan Missing Values:
                 Jumlah Hilang Persentase (%)
TrailingPE                  23          5.81%
Beta                         6          1.52%
EBITDA                       5          1.26%
EnterpriseValue              3          0.76%
TotalDebt                    3          0.76%
TaxProvision                 3          0.76%
TotalRevenue                 2          0.51%
PretaxIncome                 2          0.51%
EBIT                         2          0.51%


# Export Dataset

In [8]:
file_name = 'sp500_relative_valuation_dataset.csv'
df_final.to_csv(file_name, index=False)
print(f"File dataset disimpan ke: {file_name}")

File dataset disimpan ke: sp500_relative_valuation_dataset.csv
